# Overview

This is a notebook with over 100 features created in this competition. The cause of my failure was clear.  
・ In the first neural network competition, I stuck to feature engineering in anticipation of a score increase similar to that of lightgbm.  
・ I couldn't make a model. Obviously too complex models won't work, but transformers and simple lstm are on top.  
・ I couldn't fully understand control engineering. The mediocre and general feature I thought of was something that many could come up with, and the losses did not diminish and did not differentiate.  

I hope you can learn from my mistakes and these features will help people who are learning feature engineering.Let's do our best next time.

I learned from these notebooks. thank you very much!

https://www.kaggle.com/carlmcbrideellis/ventilator-pressure-eda-and-simple-submission  
https://www.kaggle.com/marutama/eda-about-u-in  
https://www.kaggle.com/lucasmorin/u-in-mae-exploration-with-umap-hdbscan  
https://www.kaggle.com/lucasmorin/time-series-agregation-functions  
https://www.kaggle.com/aerdem4/google-ventilator-lofo-feature-importance  https://www.kaggle.com/lucasmorin/spectral-analysis-feature-engineering



In [ ]:
train['first_value_u_in'] = train.groupby(['breath_id'])['u_in'].transform('first')
train['last_value_u_in'] = train.groupby(['breath_id'])['u_in'].transform('last')

for i in range(1,11):
    train[f'u_in_lag{i}'] = train.groupby('breath_id')['u_in'].shift(i)
train = train.fillna(0)
for i in range(1,11):
    train[f'u_in_lag{i}'] = train[f'u_in_lag{i}'].astype(np.float32)
for i in range(1,11):


for i in range(1,11):
    train[f'u_out_lag{i}'] = train.groupby('breath_id')['u_out'].shift(i)
train = train.fillna(0)
for i in range(1,11):
    train[f'u_out_lag{i}'] = train[f'u_out_lag{i}'].astype(np.float32)

# u_in_diff2to10
for i in range(1,11):
    train[f'u_in_lag{i}'] = train.groupby('breath_id')['u_in'].shift(i)
train = train.fillna(0)
for i in range(1,11):
    train[f'u_in_lagdiff{i}'] = train['u_in'] - train[f'u_in_lag{i}']
    train[f'u_in_lagdiff{i}'] = train[f'u_in_lagdiff{i}'].astype(np.float16)

for i in range(1,11):
    test[f'u_in_lag{i}'] = test.groupby('breath_id')['u_in'].shift(i)
test = test.fillna(0)
for i in range(1,11):
    test[f'u_in_lagdiff{i}'] = test['u_in'] - test[f'u_in_lag{i}']
    test[f'u_in_lagdiff{i}'] = test[f'u_in_lagdiff{i}'].astype(np.float16)


for i in range(1,11):
    train[f'u_in_lagback{-i}'] = train.groupby('breath_id')['u_in'].shift(-i)
train = train.fillna(0)
for i in range(1,11):
    train[f'u_in_lagback{-i}'] = train[f'u_in_lagback{-i}'].astype(np.float16)

for i in range(1,11):
    test[f'u_in_lagback{-i}'] = test.groupby('breath_id')['u_in'].shift(-i)
test = test.fillna(0)
for i in range(1,11):
    test[f'u_in_lagback{-i}'] = test[f'u_in_lagback{-i}'].astype(np.float16)

for i in range(1,11):
    train[f'u_in_lagback{-i}'] = train.groupby('breath_id')['u_in'].shift(-i)
train = train.fillna(0)
for i in range(1,11):
    train[f'u_in_lagbackdiff{-i}'] = train['u_in'] - train[f'u_in_lagback{-i}']
    train[f'u_in_lagbackdiff{-i}'] = train[f'u_in_lagbackdiff{-i}'].astype(np.int16)
for i in range(1,11):
    test[f'u_in_lagback{-i}'] = test.groupby('breath_id')['u_in'].shift(-i)
test = test.fillna(0)
for i in range(1,11):
    test[f'u_in_lagbackdiff{-i}'] = test['u_in'] - test[f'u_in_lagback{-i}']
    test[f'u_in_lagbackdiff{-i}'] = test[f'u_in_lagbackdiff{-i}'].astype(np.int16)


tmp=pd.DataFrame()
for j in [60,55,50,45,40,35,30,12,10,6]:
    tmp[f'time_step_binning_{j}'] = pd.cut(train['time_step'],
                    bins=j,
                    labels=[i for i in range(1, j+1)],right=True)

tmp=pd.DataFrame()
for j in [60,40,35,30,12,10]:
    tmp[f'time_step_binning_{j}'] = pd.cut(train['time_step'],
                    bins=j,
                    labels=[i for i in range(1, j+1)],right=True)
    tmp[f'time_step_binning_{j}'] = tmp[f'time_step_binning_{j}'].astype('int16')

tmp = pd.get_dummies(train['C'], prefix='C_OHE')
tmp = pd.get_dummies(train['R'], prefix='R_OHE')

def rename_multicol(df):
    df_col=df.columns #列名をコピー
    df = df.T.reset_index(drop=False).T #一回列名をリセット
    for i in range(df.shape[1]): #列名を新たに定義
        rename_col = {i:"_".join(df_col[i])}
        df = df.rename(columns = rename_col)     
    df = df.drop(["level_0","level_1"],axis=0)
    return df

tmp = train.groupby(['R','C']).agg({'pressure': ['min','max','mean','median','std']}).reset_index()
tmp = rename_multicol(tmp)
tmp = tmp.rename(columns={'R_': 'R', 'C_': 'C'})

train = train.merge(tmp, how='inner',on=['R','C'])
test = test.merge(tmp, how='inner',on=['R','C'])



tmp = train.groupby(['breath_id']).agg({'u_in': ['min','max','mean','median','std']}).reset_index()
tmp = rename_multicol(tmp)
tmp = tmp.rename(columns={'breath_id_': 'breath_id'})
train = train.merge(tmp, how='inner',on=['breath_id'])


train['breath_id_u_in_diffmax'] = train.groupby(['breath_id'])['u_in'].transform('max') - train['u_in']
train['breath_id_u_in_diffmean'] = train.groupby(['breath_id'])['u_in'].transform('mean') - train['u_in']
train['breath_id_u_in_diffmedian'] = train.groupby(['breath_id'])['u_in'].transform('median') - train['u_in']
train['breath_id_u_out_u_in_min'] = train.groupby(['breath_id','u_out'])['u_in'].transform('min').astype(np.float32)
train['breath_id_u_out_u_in_max'] = train.groupby(['breath_id','u_out'])['u_in'].transform('max').astype(np.float32)
train['breath_id_u_out_u_in_mean'] = train.groupby(['breath_id','u_out'])['u_in'].transform('mean').astype(np.float32)
train['breath_id_u_out_u_in_median'] = train.groupby(['breath_id','u_out'])['u_in'].transform('median').astype(np.float32)
train['breath_id_u_out_u_in_std'] = train.groupby(['breath_id','u_out'])['u_in'].transform('std').astype(np.float32)

tmp = pd.DataFrame()
tmp['u_in_cumsum'] = train.groupby(['breath_id'])['u_in'].cumsum()
tmp = pd.DataFrame()
tmp['u_in_cumsum'] = test.groupby(['breath_id'])['u_in'].cumsum()


train['one'] = 1
train['count'] = (train['one']).groupby(train['breath_id']).cumsum()
train['u_in_cumsum'] = train.groupby(['breath_id'])['u_in'].cumsum()
train['u_in_cummean'] =train['u_in_cumsum'] /train['count']

tmp = pd.DataFrame()
tmp['time_step_cumsum'] = train.groupby(['breath_id'])['time_step'].cumsum()
tmp = pd.DataFrame()
tmp['time_step_cumsum'] = test.groupby(['breath_id'])['time_step'].cumsum()


train['cross']= train['u_in']*train['u_out']
train['cross2']= train['time_step']*train['u_out']

def area(df):
    df['area_u_in_u_out']= df['u_in']*df['u_out']
    df['area_time_step_u_in'] = df['time_step'] * df['u_in']
    df['area_time_step_u_incumsum'] = df.groupby('breath_id')['area_time_step_u_in'].cumsum()

    for i in range(1,5):
        df[f'u_in_lag{i}'] = df.groupby('breath_id')['u_in'].shift(i)
    df = df.fillna(0)
    for i in range(1,5):
        df[f'u_in_lag{i}'] = df[f'u_in_lag{i}'].astype(np.float32)
        df[f'u_in_lagdiff{i}'] = df['u_in'] - df[f'u_in_lag{i}']

    for i in range(1,5):
        df[f'time_step_lag{i}'] = df.groupby('breath_id')['time_step'].shift(i)
    df = df.fillna(0)
    for i in range(1,5):
        df[f'time_step_lag{i}'] = df[f'time_step_lag{i}'].astype(np.float32)
        df[f'time_step_lagdiff{i}'] = df['time_step'] - df[f'time_step_lag{i}']

    for i in range(1,5):
        df[f'area_time_steplagdiff_u_inlagdiff{i}'] = df[f'time_step_lagdiff{i}'] * df[f'u_in_lagdiff{i}']

    return df


def breath_id_u_in_rolling10meanmedianmaxstd(df):
    df['breath_id_u_in_rolling_10_mean'] = train.groupby(['breath_id'])['u_in'].transform(lambda x: x.rolling(10, 1).mean())
    df['breath_id_u_in_rolling_10_median'] = train.groupby(['breath_id'])['u_in'].transform(lambda x: x.rolling(10, 1).median())
    df['breath_id_u_in_rolling_10_max'] = train.groupby(['breath_id'])['u_in'].transform(lambda x: x.rolling(10, 1).max())
    df['breath_id_u_in_rolling_10_std'] = train.groupby(['breath_id'])['u_in'].transform(lambda x: x.rolling(10, 1).std())
    return df

train = breath_id_u_in_rolling10meanmedianmaxstd(train)
train = train.fillna(0)
test = breath_id_u_in_rolling10meanmedianmaxstd(test)
test = test.fillna(0)

def breath_id_u_in_expand_meanmaxstd(df):
    df['breath_id_u_in_expand_mean'] = df.groupby('breath_id')['u_in'].expanding(2).mean().reset_index(level=0,drop=True)
    # df['breath_id_u_in_expand_median'] = df.groupby('breath_id')['u_in'].expanding(2).median().reset_index(level=0,drop=True)
    df['breath_id_u_in_expand_max'] = df.groupby('breath_id')['u_in'].expanding(2).max().reset_index(level=0,drop=True)
    df['breath_id_u_in_expand_std'] = df.groupby('breath_id')['u_in'].expanding(2).std().reset_index(level=0,drop=True)
    return df

train = breath_id_u_in_expand_meanmaxstd(train)
train = train.fillna(0)
test = breath_id_u_in_expand_meanmaxstd(test)
test = test.fillna(0)


def breath_id_u_in_ewm_u_in_meanstdcorr(df):
    df['breath_id_u_in_ewm_u_in_mean'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_corr'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).corr().reset_index(level=0,drop=True)
    return df

train = breath_id_u_in_ewm_u_in_meanstdcorr(train)
test = breath_id_u_in_ewm_u_in_meanstdcorr(test)
train = train.fillna(0)
test = test.fillna(0)


def breath_id_u_in_ewm_u_in_meanstd001to09(df):
    df['breath_id_u_in_ewm_u_in_mean'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.01'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.01,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.05'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.05,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.1'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.1,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.2'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.2,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.3'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.3,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.4'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.4,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.5'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.5,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.6'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.6,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.7'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.7,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_mean0.8'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.8,min_periods=2,adjust=False).mean().reset_index(level=0,drop=True)

    df['breath_id_u_in_ewm_u_in_std0.01'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.01,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.05'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.05,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.1'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.1,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.2'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.2,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.3'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.3,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.4'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.4,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.5'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.5,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.6'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.6,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.7'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.7,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    df['breath_id_u_in_ewm_u_in_std0.8'] = df.groupby('breath_id')['u_in'].ewm(alpha=0.8,min_periods=2,adjust=False).std().reset_index(level=0,drop=True)
    return df

train = breath_id_u_in_ewm_u_in_meanstd001to09(train)
test = breath_id_u_in_ewm_u_in_meanstd001to09(test)
train = train.fillna(0)
test = test.fillna(0)

def mktime_step_end(df):
    df['time_step_end'] = df.groupby(["breath_id"])['time_step'].transform('last').astype(np.float32)
    return df

def mk_sign_and_vib(df):
    for i in range(1,2):
        df[f'u_in_lag{i}'] = df.groupby('breath_id')['u_in'].shift(i)
    df = df.fillna(0)
    for i in range(1,2):
        df[f'u_in_lag{i}'] = df[f'u_in_lag{i}'].astype(np.float32)
        df[f'u_in_lagdiff{i}'] = df['u_in'] - df[f'u_in_lag{i}']
    first_df = df.loc[0::80,:]


    df['u_out_diff'] = df['u_out'].diff()
    df['u_out_diff'].fillna(0, inplace=True)
    df['u_out_diff'].replace(-1, 0, inplace=True)
    uout1_df = df[df['u_out_diff']==1]
    first_0_dict = dict(zip(first_df['id'], [0]*len(uout1_df)))

    df['diff_sign'] = np.sign(df['u_in_lagdiff1'])
    df['sign_diff'] = df['diff_sign'].diff()
    df['tmp'] = df['id'].map(first_0_dict)
    df.iloc[0::80, df.columns.get_loc('sign_diff')] = df.iloc[0::80, df.columns.get_loc('tmp')]
    df.drop(['tmp','u_out_diff'],axis=1,inplace=True)
    del first_0_dict

    df['sign_diff'] = abs(df['sign_diff']) 
    df['diff_vib'] = df.groupby(['breath_id'])['sign_diff'].transform('sum')

    return df

train = mk_sign_and_vib(train)
test = mk_sign_and_vib(test)



# 1013

# P alv = V / C RS + PEEP
# P alv = u_in / C + PEEP
def p_alveolar(df):
    df['u_in/C'] = df['u_in']/df['C']
    df['peep'] = df.groupby(['breath_id'])['u_in'].transform("last")
    df['P_alv'] = df['u_in/C'] + df['peep']
    return df
train = p_alveolar(train)
test = p_alveolar(test)


def p_plat(df):
    df['Vt'] = df.groupby(['breath_id'])['u_in'].transform('sum') +1e-3
    df['pip'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['peep'] = df.groupby(['breath_id'])['u_in'].transform("last")

    # ###########################################################
    # exhaled_VT
    # VEXH
    # tE 
    df['breath_id__u_in__lag1'] = df.groupby(['breath_id'])['u_in'].shift(1)
    df = df.fillna(0)
    df['u_in_lagdiff1'] = df['u_in'] - df['breath_id__u_in__lag1']
    df['minus_u_in_lagdiff1'] = df['u_in_lagdiff1'] * -1

    tmpdf = df.query('u_out==1')

    exhaled_VT = tmpdf.groupby(['breath_id'])['u_in'].agg('max')-tmpdf.groupby(['breath_id'])['u_in'].agg('min')
    exhaled_VT_dict = exhaled_VT.to_dict()
    VEXH = tmpdf.groupby(['breath_id'])['minus_u_in_lagdiff1'].max() + 1e-3
    VEXH_dict = VEXH.to_dict()

    exhaled_VT = np.array(list(exhaled_VT_dict.values()))
    VEXH = np.array(list(VEXH_dict.values()))
    tE = exhaled_VT / VEXH
    tE = pd.DataFrame(tE,index=VEXH_dict.keys())

    df = pd.merge(df, tE, left_on="breath_id", right_index=True)
    df = df.rename(columns={0: 'tE'})
    #################################################################################################
    df['Pplat'] = ((df['Vt'] * df['pip'])-(df['Vt'] * df['peep']))  /  (df['Vt'] + (df['tE'] * df['u_in']))

    return df

def p_plat_escape0(df):
    df['Vt'] = df.groupby(['breath_id'])['u_in'].transform('sum') +1e-3
    df['pip'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['peep'] = df.groupby(['breath_id'])['u_in'].transform("last")

    # ###########################################################
    # exhaled_VT
    # VEXH
    # tE 
    df['breath_id__u_in__lag1'] = df.groupby(['breath_id'])['u_in'].shift(1)
    df = df.fillna(0)
    df['u_in_lagdiff1'] = df['u_in'] - df['breath_id__u_in__lag1']
    df['minus_u_in_lagdiff1'] = df['u_in_lagdiff1'] * -1

    tmpdf = df.query('u_out==1')

    exhaled_VT = tmpdf.groupby(['breath_id'])['u_in'].agg('max')-tmpdf.groupby(['breath_id'])['u_in'].agg('min')
    exhaled_VT_dict = exhaled_VT.to_dict()
    VEXH = tmpdf.groupby(['breath_id'])['minus_u_in_lagdiff1'].max() + 1e-3
    VEXH_dict = VEXH.to_dict()

    exhaled_VT = np.array(list(exhaled_VT_dict.values()))
    VEXH = np.array(list(VEXH_dict.values()))
    tE = exhaled_VT / VEXH
    tE = pd.DataFrame(tE,index=VEXH_dict.keys())

    df = pd.merge(df, tE, left_on="breath_id", right_index=True)
    df = df.rename(columns={0: 'tE'})
    #################################################################################################
    df['Vt*pip'] = (df['Vt'] * df['pip'])
    df['-(Vt-peep)'] = -(df['Vt'] * df['peep'])
    df['Vt+(tE*Vi)'] = (df['Vt'] + (df['tE'] * df['u_in']))

    return df

train['u_in_lag1'] = train.groupby('breath_id')['u_in'].shift(1)
train = train.fillna(0)
train['u_in_lagdiff1'] = train['u_in'] - train['u_in_lag1']
train['pressurediff'] = train.groupby('breath_id')['pressure'].diff(1)
train = train.fillna(0)
train['Pseudo_change'] = train['pressurediff']/train['u_in_lagdiff1'] # 圧力の変化に対するu_iNの変化
train = train.fillna(0)
train = train.replace({np.inf:0,-np.inf:0})

def rename_multicol(df):
            df_col=df.columns #列名をコピー
            df = df.T.reset_index(drop=False).T #一回列名をリセット
            for i in range(df.shape[1]): #列名を新たに定義
                rename_col = {i:"_".join(df_col[i])}
                df = df.rename(columns = rename_col)     
            df = df.drop(["level_0","level_1"],axis=0)
            return df

tmpc = train.groupby(['C']).agg({'Pseudo_change': ['min','max','mean','std']}).reset_index()
tmp = rename_multicol(tmpc)
tmp = tmp.rename(columns={'C_': 'C'})
train = train.merge(tmp, how='inner',on=['C'])
train = train.rename(
    {"Pseudo_change_min":"C_Pseudo_change_min",
        "Pseudo_change_max":"C_Pseudo_change_max",
        "Pseudo_change_mean":"C_Pseudo_change_mean",
        "Pseudo_change_std":"C_Pseudo_change_std"},axis=1)

tmpr = train.groupby(['R']).agg({'Pseudo_change': ['min','max','mean','std']}).reset_index()
tmp = rename_multicol(tmpr)
tmp = tmp.rename(columns={'R_': 'R'})
train = train.merge(tmp, how='inner',on=['R'])
train = train.rename(
    {"Pseudo_change_min":"R_Pseudo_change_min",
        "Pseudo_change_max":"R_Pseudo_change_max",
        "Pseudo_change_mean":"R_Pseudo_change_mean",
        "Pseudo_change_std":"R_Pseudo_change_std"},axis=1)


def u_in_in_second(df):
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df = df.fillna(0)
    df['u_in_lagdiff1'] = df['u_in'] - df['u_in_lag1']
    df['u_in_lagdiff1'] = df['u_in_lagdiff1'].astype(np.float16)

    df['time_step_lag1'] = df.groupby('breath_id')['time_step'].shift(1)
    df = df.fillna(0)
    df['time_step_lagdiff1'] = df['time_step'] - df['time_step_lag1']
    df['time_step_lagdiff1'] = df['time_step_lagdiff1'].astype(np.float16)
    df['u_in/s'] = df['u_in_lagdiff1']/df['time_step_lagdiff1']
    df = df.fillna(0)
    df = df.replace({np.inf:0,-np.inf:0})
    return df

def peaknum(df):
    user = df.breath_id.unique()
    peaklist=[]
    for i in tqdm(user):
        df_ = df.query(f'breath_id == {i}')
        ispeak = []
        for j in range(0,80):
            # print(np.max(df_['u_in'][:j+1].values)==df_['u_in'].values[j])
            ispeak.append(np.max(df_['u_in'][:j+1].values)==df_['u_in'].values[j])
        peaklist.extend(list(np.cumsum(ispeak)))
    return peaklist

def differential_10(df):
    window=15
    from tqdm import tqdm
    user = df.breath_id.unique()

    window=10
    list10 = []
    for i in tqdm(user):
        df_ = df.query(f'breath_id =={i}')
        numrow = len(df_.query('u_out==0'))
        tmp = np.zeros((80))
        j=0
        while j < 40:
            tmp[j] = ((df_['u_in'].values[j+window]-df_['u_in'].values[j]))
            # print((df_['u_in'].values[j+window],df_['u_in'].values[j]))
            j+=1
        tmp[numrow:]=0
        list10.extend(list(tmp))

    df['differential10'] = pd.DataFrame(list10)
    return df

train =  differential_10(train)
test =  differential_10(test)



def differential_10(df):
    window=15
    from tqdm import tqdm
    user = df.breath_id.unique()

    # df['differential15'] = pd.DataFrame(list15)
    df['differential10'] = pd.DataFrame(list10)
    # df['differential5'] = pd.DataFrame(list5)
    return df

def rolling_2to10_summaxmean(df):
    print('add features...')
    df[["rolling_2_in_sum","rolling_2_in_max","rolling_2_in_mean"]] = \
        (df.groupby('breath_id')['u_in'].rolling(window=2,min_periods=1)\
                                        .agg({"rolling_2_in_sum":"sum",
                                            "rolling_2_in_max":"max",
                                            "rolling_2_in_mean":"mean"})\
                                        .reset_index(level=0,drop=True))
    df[["rolling_4_in_sum","rolling_4_in_max","rolling_4_in_mean"]] = \
        (df.groupby('breath_id')['u_in'].rolling(window=4,min_periods=1)\
                                        .agg({"rolling_4_in_sum":"sum",
                                            "rolling_4_in_max":"max",
                                            "rolling_4_in_mean":"mean"})\
                                        .reset_index(level=0,drop=True))
    df[["rolling_6_in_sum","rolling_6_in_max","rolling_6_in_mean"]] = \
        (df.groupby('breath_id')['u_in'].rolling(window=6,min_periods=1)\
                                        .agg({"rolling_6_in_sum":"sum",
                                            "rolling_6_in_max":"max",
                                            "rolling_6_in_mean":"mean"})\
                                        .reset_index(level=0,drop=True))
    df[["rolling_8_in_sum","rolling_8_in_max","rolling_8_in_mean"]] = \
        (df.groupby('breath_id')['u_in'].rolling(window=8,min_periods=1)\
                                        .agg({"rolling_8_in_sum":"sum",
                                            "rolling_8_in_max":"max",
                                            "rolling_8_in_mean":"mean"})\
                                        .reset_index(level=0,drop=True))
    df[["rolling_10_in_sum","rolling_10_in_max","rolling_10_in_mean"]] = \
        (df.groupby('breath_id')['u_in'].rolling(window=10,min_periods=1)\
                                        .agg({"rolling_10_in_sum":"sum",

                                            "rolling_10_in_max":"max",
                                            "rolling_10_in_mean":"mean"})\
                                        .reset_index(level=0,drop=True))
    print('add features finished')
    return df


# from https://www.kaggle.com/lucasmorin/spectral-analysis-feature-engineering
def ffts(df):
    N = 80
    w = blackman(N+1)
    ffta = lambda x: np.abs(fft(np.append(x.values,x.values[0]))[:80])
    ffta.__name__ = 'ffta'

    fftw = lambda x: np.abs(fft(np.append(x.values,x.values[0])*w)[:80])
    fftw.__name__ = 'fftw'

    df['fft_u_in'] = df.groupby('breath_id')['u_in'].transform(ffta)
    df['fft_u_in_w'] = df.groupby('breath_id')['u_in'].transform(fftw)
    df['analytical'] = abs(df.groupby('breath_id')['u_in'].transform(hilbert))
    df['envelope'] = np.abs(df['analytical'])
    df['phase'] = np.angle(df['analytical'])
    df['unwrapped_phase'] = df.groupby('breath_id')['phase'].transform(np.unwrap)
    df['phase_shift1'] = df.groupby('breath_id')['unwrapped_phase'].shift(1).astype(np.float32)
    df['IF'] = df['unwrapped_phase'] - df['phase_shift1'].astype(np.float32)
    df = df.fillna(0)
    return df